In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
from joblib import Parallel, delayed
from collections import defaultdict, OrderedDict
import json
import re
import tensorflow as tf

In [8]:
ratings = pd.read_table('data/ratings.dat', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], sep='::')
users = pd.read_table('data/users.dat', header=None, names=['UserID','Gender','Age','Occupation','Zip-code'], sep='::')
movies = pd.read_table('data/movies.dat', header=None, names=['MovieID', 'Title', 'Genres'], sep='::')
data = pd.merge(pd.merge(users, ratings), movies)
data1 = data.drop(['Gender','Age','Occupation','Zip-code','Timestamp','Title'],1,inplace=False)
data2 = data.drop(['Title','Zip-code','Timestamp'],1,inplace=False)

genres_unique = pd.DataFrame(data1['Genres'].str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['Genre'])
data1 = data1.join(data1.Genres.str.get_dummies())
data1.drop('Genres', inplace=True, axis=1)

data2.Gender = data2.Gender.apply(lambda x: 1 if x=='F' else 0)
data2 = data2.drop(['UserID','MovieID','Genres','Rating'],1)
data1 = pd.concat([data2,data1],1)

order =['Gender', 'Age', 'Occupation', 'Action','Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
        ,'UserID', 'MovieID', 'Rating']
data1 = data1[order]

data1.columns=['Gender', 'Age', 'Occupation', 'Action', 'Adventure', 'Animation',
       "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western', 'user_id', 'item_id', 'target']
train, test = train_test_split(data1,test_size=0.2)

train.to_csv('data/train.csv',index=None)
test.to_csv('data/test.csv',index=None)